### Задача
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [26]:
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind

In [27]:
# Чтение данных из Excel файла
df = pd.read_excel('AB_Test_Results.xlsx')

C:\Users\Sarmat\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [28]:
# Первичный анализ данных
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [29]:
# Первичный анализ данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [30]:
# Первичный анализ данных
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [31]:
# Первичный анализ данных
df.isnull().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [32]:
# Преобразование столбца 'VARIANT_NAME' в dummy переменные
df = pd.get_dummies(df, columns=['VARIANT_NAME'])

In [33]:
# Разделение данных на контрольную и экспериментальную группы
control_group = df[df['VARIANT_NAME_control'] == 1]
test_group = df[df['VARIANT_NAME_variant'] == 1]

# Проверка размера групп
print(f"Размер контрольной группы: {len(control_group)}")
print(f"Размер экспериментальной группы: {len(test_group)}")

Размер контрольной группы: 4984
Размер экспериментальной группы: 5016


In [34]:
# Функция для проверки нормальности распределения
def check_normality(group, group_name):
    if len(group) >= 3:
        stat, p_value = shapiro(group['REVENUE'])
        print(f'{group_name} - Shapiro-Wilk Test: Statistics={stat:.3f}, p={p_value:.3f}')
        return p_value
    else:
        print(f'{group_name} содержит менее 3 элементов. Невозможно провести тест Шапиро-Уилка.')
        return None

# Проверка нормальности для контрольной группы
p_value_control = check_normality(control_group, 'Control Group')

# Проверка нормальности для экспериментальной группы
p_value_test = check_normality(test_group, 'Test Group')

# Проверка гомогенности дисперсий (если обе группы нормальны)
if p_value_control is not None and p_value_control > 0.05 and p_value_test is not None and p_value_test > 0.05:
    stat, p_value = levene(control_group['REVENUE'], test_group['REVENUE'])
    print('Levene Test: Statistics=%.3f, p=%.3f' % (stat, p_value))

    # Применение t-теста для независимых выборок
    stat, p_value = ttest_ind(control_group['REVENUE'], test_group['REVENUE'], equal_var=(p_value > 0.05))
    print('t-test: Statistics=%.3f, p=%.3f' % (stat, p_value))

    # Формулирование рекомендаций
    if p_value < 0.05:
        recommendation = "Изменения в доходе статистически значимы. Рекомендуется внедрить изменения на постоянной основе."
    else:
        recommendation = "Изменения в доходе не являются статистически значимыми. Рекомендуется провести дополнительные тесты или пересмотреть гипотезу."
else:
    recommendation = "Не удалось провести корректный статистический анализ из-за недостаточного количества данных или ненормальности распределения."

print(recommendation)

Control Group - Shapiro-Wilk Test: Statistics=0.018, p=0.000
Test Group - Shapiro-Wilk Test: Statistics=0.027, p=0.000
Не удалось провести корректный статистический анализ из-за недостаточного количества данных или ненормальности распределения.


C:\Users\Sarmat\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
